## Computer vision: full body capture

### Introduction

To demonstrate more sophisticated computer vision, we will be using Google's MediaPipe library to pick up full-body movement, everything from facial expressions and hand waving to your overall stance. MediaPipe is machine learning toolkit that lets a computer "see" things as they happen, with hardly any extra code on our part. It runs a fast deep learning pipeline that can recognise a face, track a hand, map the joints in your body or spot everyday objects all on a mobile phone or other small device, rather than a traditional desktop. To reach this accuracy, MediaPipe was trained on vast collections of real photos plus lots of cleverly generated images. The researchers built 3D models of hands and bodies, altered the lighting, turned them around and even hid bits from view, so the system would cope with messy and unpredictable real-world scenarios. In this practical we will look at a few models used to capture faces and full body motion using MediaPipe.

### Face capture

MediaPipe's Face Mesh was not built from scratch, but trained on millions of labelled pictures, the main one being the *300W-LP* dataset. Think of this as a vast family album in which every face already has 468 tiny dots drawn on it to mark key features like eyes, eyebrows, nose, lips, and jawline. These landmarks act as anchor points so the model can learn where a face begins and ends, regardless of pose or expression.  

Manually labelling that many images would take an impossible amount of time, so researchers turned to synthetic data. Using 3D software such as *Blender* or *Unity*, they created digital avatars and generated endless variations. Faces were rotated at different angles, lit from unusual directions, warped slightly to reflect real-world diversity, and partially occluded with glasses, hats or hands.  

To bridge the visual gap between synthetic and real faces, the team used a technique called *style transfer*. This process blends the textures, lighting and colour of synthetic images with the natural imperfections of real photographs, making them more convincing for training. For example, a computer-generated face might be overlaid with the photographic "style" of a real person in a similar pose, helping the model learn how actual faces behave in context.  

The model draws from more than one dataset. In addition to *300W-LP*, it trains on *WFLW* (a dataset with extreme facial expressions, occlusions, and odd lighting conditions) and *AFLW* (which includes faces from various ethnicities and shot from different angles, including side views and tilted heads). This broadens the model's understanding of human variation.  

The neural network is a convolutional architecture (CNN) that first detects low-level features like edges and contours. These are gradually combined into higher-level features such as eyes and mouth shapes. The model is then refined through *transfer learning* where it starts with weights borrowed from other face-recognition models and is then fine-tuned using higher-resolution, task-specific data to boost accuracy and generalisation.  

To track a face in motion, Face Mesh locates landmarks in a single frame, and also uses *temporal smoothing* to ensure the points move naturally from one frame to the next, reducing flickering and jitter. It also estimates *depth* and *surface geometry*, creating a 3D mesh that deforms as your face moves. This allows real-time applications such as to animate masks, avatars, or filters that follow your expressions precisely.  

The result is a face-mesh model that can run smoothly on everyday devices like smartphones. Thanks to the diverse training data from multiple sources, you can turn your head, squint, smile, or half-hide yourself behind a coffee cup, and it will still be able to keep up in real time.  

<p><img src="https://choyingw.github.io/works/SynergyNet/img/teaser.png" style="border-radius:3px;"></p>  
<small>Credit: Cho-Ying Wu, Qiangeng Xu and Ulrich Neumann, “Accurate 3-D Facial Geometry Prediction by Multi-Task, Multi-Modal, and Multi-Representation Landmark Refinement Network”, 2021.</small>

### Pose capture

To teach a machine to track the human body, you need thousands of images showing people in all sorts of positions, such as walking, sitting, stretching, dancing, or reaching awkwardly for something on a shelf. One of the most widely used resources for this is the *MPII Human Pose* dataset, built by the Max Planck Institute for Informatics:

<div style="display: flex; flex-wrap: wrap; gap: 10px;">
  <img src="https://www.mpi-inf.mpg.de/fileadmin/_processed_/6/8/csm_imgidx_0047_sticks_c5071f306a.png" style="width: 33%; border-radius: 3px;">
  <img src="https://www.mpi-inf.mpg.de/fileadmin/_processed_/9/1/csm_imgidx_0166_sticks_f94b3461c5.png" style="width: 33%; border-radius: 3px;">
  <img src="https://www.mpi-inf.mpg.de/fileadmin/_processed_/9/e/csm_imgidx_0475_sticks_b6ee1a675a.png" style="width: 30%; border-radius: 3px;">
<small>Credit: <a href="https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-models">MPII Human Pose dataset</a>, Max Planck Institute for Informatics.
</small>
</div>

Each image in this dataset includes detailed annotations of key joints including shoulders, elbows, wrists, hips, knees and ankles, along with their exact location in 3D space. These are called *landmarks*, and they form the skeleton that MediaPipe uses to map the body. Here's a small sample of what a few frames might look like when processed:

| Landmark | Type            | Position (X, Y, Z)                 | InFrameLikelihood |
|----------|-----------------|------------------------------------|-------------------|
| 11       | LEFT_SHOULDER  | (734.97, 550.79, -118.12)          | 0.9999            |
| 12       | RIGHT_SHOULDER | (391.27, 583.25, -321.16)          | 0.9999            |
| 13       | LEFT_ELBOW     | (903.84, 754.68, -219.67)          | 0.9836            |
| 14       | RIGHT_ELBOW    | (322.18, 842.60, -179.29)          | 0.9997            |
| 15       | LEFT_WRIST     | (1073.90, 654.97, -820.93)         | 0.9738            |
| 16       | RIGHT_WRIST    | (218.28, 1015.70, -683.66)         | 0.9956            |
| 17       | LEFT_PINKY     | (1146.16, 609.64, -957.00)         | 0.9527            |
| 18       | RIGHT_PINKY    | (176.18, 1065.84, -776.50)         | 0.9785            |

Each point comes with a *likelihood* value, which tells the system how confident it is that the body part was correctly identified and visible in frame.

To cope with the variety of human movement, the model has to recognise bodies from many angles, in all sorts of clothing and lighting conditions, and with parts of the body often out of view. Just like Face Mesh, this can be solved by combining multiple datasets, like *MPII with synthetic augmentation*, as well as, generating digital humans in 3D environments using tools like *Blender* to expand the dataset size through augmentation. Again, these virtual figures are posed, lit, distorted, and partially obscured to simulate real-life scenarios.

### The MediaPipe architecture
MediaPipe's models are built using convolutional neural networks (CNNs) to handle both feature extraction and keypoint detection. Once the raw data is gathered, the neural network is trained to estimate the full body pose from a single image. It learns not just where limbs are, but how they are likely to bend and move together, using biomechanical constraints to keep the motion plausible from frame to frame.  Typically, the neural network will be a *stacked hourglass* or *pose-residual* CNN. 

#### Stacked hourglass network

The stacked hourglass architecture is designed to spot patterns at multiple scales from fine details like fingers and eyes to whole-body posture. You can visualise the architecture of this network as an hourglass shape. The image data is gradually shrunk down to a small, compact version (like sand funnelling through the middle), then expanded back up again. This process is called *downsampling* and *upsampling*. It allows the network to first capture the big picture (rough body layout) and then fill in the detail (precise joint positions).

Each hourglass is like a cycle of zooming out and zooming in, the network stacks several of these in a row (hence stacked hourglass). Each stack refines the pose estimate based on the output of the previous one. This provides us with a way of progressively polishing the guess, layer by layer.  The advantage of this type of network is that it captures both global context and local detail, and using multiple hourglasses allows step-by-step refinement. This technique is often used in high-resolution pose estimation tasks.

#### Pose-residual network (PRN)

A pose-residual network starts with a rough prediction of where each joint might be, then learns how to correct its own mistakes. It's a bit like sketching out a pose quickly, and then nudging each point until everything looks more anatomically plausible.

The idea is to model not just individual joints, but their relationships. For instance, if your left elbow moves, your left wrist probably has to follow. The network learns these correlations by feeding back into itself and applying residual updates with small corrections that improve the estimated pose. This approach is especially good for tuning the layout of the joints after an initial guess. It adds realism and smoothness, helping to avoid unnatural results like elbows bending backwards or limbs floating in mid-air. The main advantage of this is that it learns joint relationships and dependencies, and makes corrections through residual feedback to ensure anatomical realism.

These techniques are often combined with temporal smoothing for video (to stop landmarks from flickering) and with 3D geometry models to map depth and rotation. In addition, these models are tuned to balance speed and accuracy so that it can work in real time.   This makes it ideal for use on mobile phones or embedded systems, where it can be used for gesture interfaces, fitness apps, augmented reality, or even sign language interpretation.

Once a region of interest (such as a face or hand) is identified, MediaPipe (or any vision model) tries to work out the exact location of a landmark, like the tip of your nose, your left wrist, or the base of your thumb. To achieve this it often uses two complementary techniques known as *heatmaps* and *regression heads*.

#### Heatmaps

A heatmap is like a glowing spotlight on an image that shows where the model thinks a particular landmark might be. The brighter (or hotter) the spot, the more confident the model is that the point is in that area. Rather than guessing a single coordinate directly, the model produces a low-resolution 2D map for each keypoint. These maps show likelihoods, allowing the system to factor in uncertainty and context (e.g. this is probably an eye, but if not, it might be just above, or to the left...). This method works well because it provides spatial cues and helps when landmarks are close together. It also allows the model to "see" nearby features when making a decision, and it is robust to small occlusions or blurs in the image. However, heatmaps only give coarse estimates and need further refinement.

#### Regression heads

This is where regression heads come in. Once a rough position is identified, the regression head fine-tunes it. This part of the model directly predicts the exact `(x, y)` or `(x, y, z)` coordinates of the landmark based on the extracted features. In other words, heatmaps give a good guess of where the keypoint is, whereas, regression heads zoom in and state "its exactly at this point".  Combining both techniques gives the best of both worlds giving broad, flexible location from heatmaps, and precise final placement from regression.

This hybrid approach makes MediaPipe's landmark detection fast and accurate, even when working in real-world conditions. These are further supported by lightweight model architectures such as *MobileNetV2*, *BlazeFace*, and *BlazePose*, all of which are optimised for real-time performance. We will discuss these briefly, but it is worth reading up on some of them if you want to learn more details about their architecture (see links below).

- *Face Mesh* (468 facial landmarks) begins with <a href="https://ai.google.dev/edge/mediapipe/solutions/vision/face_detector", target="_blank">*BlazeFace*</a> to detect the presence and rough location of the face. A separate refinement model then predicts a high-resolution mesh of landmark points across the facial surface.

- *Pose Estimation* (33 keypoints across the body) uses <a href="https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker" target="_blank">*BlazePose*</a>, a fast and small CNN model trained to locate major joints such as shoulders, elbows, hips and knees.

- *Hand Tracking* (21 landmarks per hand) uses a model called <a href="https://research.google/blog/on-device-real-time-hand-tracking-with-mediapipe/" target="_blank">*BlazePalm*</a> to detect hand regions, followed by a specialised landmark model to refine the position of each finger joint.

To keep inference times low and make these models practical on phones etc., MediaPipe uses a cascade approach, where a small detection model first identifies the approximate location of the object (e.g. a bounding box around a face or hand). A second, more detailed model then refines this prediction by extracting accurate landmarks. This two-stage design involving a *detection* stage followed by a *refinement* stage allows for high-speed processing without sacrificing too much accuracy.

### Installing Python libraries

In [ ]:
!pip install opencv-python mediapipe numpy==1.24.4 # Mediapipe prefers this version of numpy

First, we import the necessary Python libraries and set up the Holistic model. Mediapipe is the library from Google that provides pre-trained models for detecting face, hands, and body landmarks. 

Our code will capture video using OpenCV (from a webcam), and use MediaPipe's model to detect the face, pose, and hands. We will then draw the detected landmarks and connections onto the video feed and display it (for visualisation).

The library is modular, so you can access specific models via `mp.solutions`, such as `pose`, `hands`, `face_mesh`, and in our case `holistic`. This is a wrapper that brings together face landmarks, hand landmarks, and body pose estimation into a single pipeline, which will allow us to track everything in one go, rather than running three separate models:

In [ ]:
import cv2
import mediapipe as mp

# Initialise the model
mp_holistic = mp.solutions.holistic

# Instantiate full body capture (using default parameters)
holistic = mp_holistic.Holistic()

We have now instantiated the model. When creating the holistic model, you can customise its behaviour by passing in several optional parameters. For example:

```
mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    refine_face_landmarks=True
)
```
These settings control how the model processes input. Setting `static_image_mode=True` treats each input as a standalone image, useful for photos rather than video streams. The `model_complexity` value (0, 1, or 2) adjusts the balance between speed and accuracy, with higher values giving better results but requiring more processing power. 

Enabling `smooth_landmarks` helps stabilise predictions over time by reducing jitter between frames. If `enable_segmentation=True`, the model also outputs a rough outline of the person, separating them from the background. Finally, `refine_face_landmarks=True` adds extra precision around the eyes, eyebrows and lips ideal for applications where facial detail matters, such as avatar animation/bunny ears or expression tracking.

We next define our own custom styles for a better visualisation. We make use of `mp_drawing`, which provides functions to draw landmarks and the lines (connections) between them directly onto an image or video frame, for example, drawing skeleton joints or face meshes.

We also use `mp_drawing_styles`, which includes built-in style presets with consistent colours and thicknesses for hands, faces, and body poses. Here we amend the thickness of the lines drawn to the feed, as well as, the size of points/landmarks.  We also specify the colour of each to distinguish between them:

In [ ]:
# Import drawing utilities and pre-defined visual styles from MediaPipe
mp_drawing = mp.solutions.drawing_utils          # Provides functions for rendering landmarks and connections
mp_drawing_styles = mp.solutions.drawing_styles  # Contains built-in style presets for face, pose, and hand landmarks

# Define custom styles for landmarks and their connections
# These styles control how the output is visualised on the video frame

# Style for individual landmarks: green circles with radius 1
landmark_style = mp_drawing.DrawingSpec(
    color=(0, 255, 0),        # Bright green (in BGR format)
    thickness=1,              # Outline thickness of the circle
    circle_radius=1           # Radius of the landmark point
)

# Style for visible connections between landmarks: green lines
connection_style_green = mp_drawing.DrawingSpec(
    color=(0, 255, 0),        # Green lines
    thickness=1,              # Line thickness
    circle_radius=0.5         # Optional circle at connection points
)

# Style for hidden or secondary connections: grey/white lines
connection_style_grey = mp_drawing.DrawingSpec(
    color=(255, 255, 255),    # Light grey/white lines
    thickness=1,
    circle_radius=0.5
)


### Real-time video capture

We set up a connection to our web camera (make sure you have one plugged in if not built-in) and define the viewable area of the output.

The process begins by accessing the default camera with `cv2.VideoCapture(0)`, which sets up a live video feed. An OpenCV window is created to display the results, and is configured to support fullscreen rendering.

Inside the main loop, each frame from the webcam is read using `cap.read()`. The captured frame is then passed to `holistic.process(frame)`, which runs the full-body tracking pipeline. This model detects the 468 face landmarks, 33 body pose landmarks, and 21 keypoints for each hand.

Each group of landmarks is then drawn on the video frame using `mp_drawing.draw_landmarks()`. For the face, the script uses `FACEMESH_TESSELATION` to draw a detailed mesh across the facial surface. The body landmarks are connected using `POSE_CONNECTIONS`, while the hands are visualised with `HAND_CONNECTIONS`. Custom drawing styles are applied with green lines for the face, white-grey for pose and hands, using `DrawingSpec` settings to control colour, thickness and marker size. Lastly, we use MediaPipe's default hand landmark style.

The annotated frame is displayed in the output window using `cv2.imshow()`, showing a real-time feed with facial expressions, hand gestures and body posture overlaid. This demonstrates that you can fully customise the visual output once you have the extracted landmarks, so you could add some bunny ears at this point, although we do not cover this in the practical.

The loop will continue running until we press 'q' key (quit). When the session ends, the script calls `cap.release()` to close your web camera, and `cv2.destroyAllWindows()` to shut down the display window and clean up system resources:

In [ ]:
# Get the default video device (usually the built-in webcam, device ID 0)
cap = cv2.VideoCapture(0) # 0 built-in webcam, or 1 for external camera

# Create a named OpenCV window and enable fullscreen display mode
cv2.namedWindow('MediaPipe Face, Pose, and Gesture', cv2.WND_PROP_FULLSCREEN)

# Continue while the video capture device is open and functional
while cap.isOpened():
    # Capture a single frame from the webcam
    ret, frame = cap.read()

    # If the frame is not read successfully (e.g. no camera), exit the loop
    if not ret:
        break

    # Pass the frame through the holistic model to detect face, pose, and hands
    holistic_results = holistic.process(frame)

    # Draw face landmarks (468-point mesh) using tessellation style
    mp_drawing.draw_landmarks(
        frame,
        holistic_results.face_landmarks,                     # Landmark data for the face
        mp_holistic.FACEMESH_TESSELATION,                    # Connection map for the face mesh
        landmark_drawing_spec = landmark_style,              # Green circle markers for landmarks
        connection_drawing_spec = connection_style_green     # Green lines connecting landmarks
    )

    # Draw pose landmarks (33 body keypoints) and skeletal connections
    mp_drawing.draw_landmarks(
        frame,
        holistic_results.pose_landmarks,                     # Landmark data for full-body pose
        mp_holistic.POSE_CONNECTIONS,                        # Connection map for limbs and joints
        landmark_style,                                      # Green circle markers
        connection_drawing_spec = connection_style_grey      # Grey lines for limb connections
    )

    # Draw landmarks for the left hand
    mp_drawing.draw_landmarks(
        frame,
        holistic_results.left_hand_landmarks,                # Left hand keypoints
        mp_holistic.HAND_CONNECTIONS,                        # Finger bone connections
        landmark_drawing_spec = mp_drawing_styles.get_default_hand_landmarks_style(),  # Default style
        connection_drawing_spec = connection_style_grey      # Grey connections for consistency
    )

    # Draw landmarks for the right hand
    mp_drawing.draw_landmarks(
        frame,
        holistic_results.right_hand_landmarks,               # Right hand keypoints
        mp_holistic.HAND_CONNECTIONS,                        # Finger bone connections
        landmark_drawing_spec = mp_drawing_styles.get_default_hand_landmarks_style(),  # Default style
        connection_drawing_spec = connection_style_grey      # Grey connections
    )

    # Show the annotated video frame in the display window
    # It might be displayed beneath another window, so you may need to switch windows to view.
    cv2.imshow('MediaPipe Face, Pose, and Gesture', frame)

    # Wait for 1 ms and check if the 'q' key has been pressed to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

As you can see, MediaPipe is quite lightweight and is an efficient deep learning framework designed for real-time computer vision applications, including face mesh, pose estimation, hand tracking, and object detection. 

It can be used in augmented reality (AR), fitness tracking, gaming, healthcare, and accessibility applications, such as, gaze activated actions. It also has the potential for sign language transcription if we have enough images of varying signs to train it with. The key advantage is in its ability to run on low-power devices like mobile phones and embedded systems.

### What have we learnt?

We've explored how MediaPipe uses a combination of smart design choices and training techniques to achieve accurate, real-time tracking of the face, hands, and body even on modest hardware. Rather than relying on a single dataset or model, MediaPipe integrates a layered approach that draws from both real and synthetic data, supported by efficient architectures. 

Because it's difficult and time-consuming to label thousands of images by hand, researchers use 3D rendering tools like *Blender* and *Unity* to create artificial data. These models simulate lighting, occlusion, movement and variety in pose, which are then made to resemble real photographs through style transfer. This dramatically expands the diversity of training data without needing to manually annotate each frame.

MediaPipe models start with weights pre-trained on general-purpose datasets, then fine-tune on high-resolution, task-specific data. This makes training faster, improves generalisation, and avoids overfitting to small datasets.

To make the most of unlabelled data, MediaPipe also uses self-supervised learning. For example, it can learn to predict missing landmarks based on visible ones, reducing dependence on large labelled datasets and enabling it to improve from partial or noisy examples.

In short, combining real-world datasets with synthetic augmentation, style adaptation, transfer learning and self-supervised techniques, allows us to create robust models that work accurately and efficiently, like MediaPipe, even in dynamic, unpredictable environments. MediaPipe is a good example that shows how thoughtful design can create high-end computer vision models, and how combining multiple models can bring huge benefits. These approaches are worth remembering as there may be creative ways for you to get performance out of your models just by rethinking the approach.